In [20]:
from pyquil import Program, get_qc
import qulacs as q
from qulacs.gate import *
from qulacs import PauliOperator
from qulacs.quantum_operator import create_quantum_operator_from_openfermion_file
from qulacs.quantum_operator import create_quantum_operator_from_openfermion_text
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.transforms import get_sparse_operator
from qulacs.observable import create_split_observable, create_observable_from_openfermion_text
from qulacs import Observable, QuantumStateGpu, QuantumCircuit
import pyquil.api as api
import numpy as np
import math

The process of creating a hamiltonian in qulacs is much less straightforward than with pyquil. Qulacs uses openfermion text to create the hamiltonian, which is why this cell block is much longer. It can also be read in from a text file.

In [21]:
# hopping term: a_p^dagger a_q
np.random.seed(seed=0)
U1 = 1
U2 = 0
e1 = 2
e2 = 0
n_qubits = 4
depth = 4
hamt_text = """
(-0.5 + 0j) [X0 X1] + 
(-0.5 + 0j) [X2 X3] + 
(-0.5 + 0j) [Y0 Y1] + 
(-0.5 + 0j) [Y2 Y3]
"""

hamt = create_quantum_operator_from_openfermion_text(hamt_text)

# interaction term: a_p^dagger a_q^dagger a_q a_p
hamu_text = """
(0.25 + 0j) [I3] - 
(0.25 + 0j) [Z0] - 
(0.25 + 0j) [Z2] + 
(0.25 + 0j) [Z0 Z2] + 
(0.0 + 0j) [I0] - 
(0.0 + 0j) [Z1] - 
(0.0 + 0j) [Z3] +
(0.0 + 0j) [Z1 Z3]
"""
hamU = create_quantum_operator_from_openfermion_text(hamu_text)

# number term: a_p^dagger a_p
hame_text = """
(2.0 + 0j) [I] - 
(1.0 + 0j) [Z0] - 
(1.0 + 0j) [Z2] - 
(0.0 + 0j) [I] -
(0.0 + 0j) [Z1] -
(0.0 + 0j) [Z3]
"""
hame = create_quantum_operator_from_openfermion_text(hame_text)

hamiltonian_text = """
(-0.5+0j) [X0 X1] + 
(-0.5+0j) [X2 X3] + 
(-0.5+0j) [Y0 Y1] + 
(-0.5+0j) [Y2 Y3]
(0.25+0j) [I3] - 
(0.25+0j) [Z0] - 
(0.25+0j) [Z2] + 
(0.25+0j) [Z0 Z2] + 
(0.0+0j) [I0] - 
(0.0+0j) [Z1] - 
(0.0+0j) [Z3] +
(0.0+0j) [Z1 Z3]
(2.0+0j) [I] - 
(1.0+0j) [Z0] - 
(1.0+0j) [Z2] - 
(0.0+0j) [I] -
(0.0+0j) [Z1] -
(0.0+0j) [Z3]
"""
hamiltonian = create_observable_from_openfermion_text(hamiltonian_text)
print(hamiltonian.get_term_count())

18


In [22]:
"""
from grove.pyvqe.vqe import VQE
from scipy.optimize import minimize
import numpy as np

vqe_inst = VQE(minimizer=minimize, minimizer_kwargs={'method': 'nelder-mead'})
"""

"\nfrom grove.pyvqe.vqe import VQE\nfrom scipy.optimize import minimize\nimport numpy as np\n\nvqe_inst = VQE(minimizer=minimize, minimizer_kwargs={'method': 'nelder-mead'})\n"

In [23]:
"""from qucochemistry.utils import pyquilpauli_to_qubitop
from openfermion.transforms import get_sparse_operator
h = get_sparse_operator(pyquilpauli_to_qubitop(hamiltonian))
np.savetxt('h.txt', np.real(h.todense()), '%-2d', ', ')
if h.shape[0] > 1024:
    [w, _] = sp.sparse.linalg.eigsh(h, k=1)
else:
    [w, _] = np.linalg.eigh(h.todense())
w
"""

"from qucochemistry.utils import pyquilpauli_to_qubitop\nfrom openfermion.transforms import get_sparse_operator\nh = get_sparse_operator(pyquilpauli_to_qubitop(hamiltonian))\nnp.savetxt('h.txt', np.real(h.todense()), '%-2d', ', ')\nif h.shape[0] > 1024:\n    [w, _] = sp.sparse.linalg.eigsh(h, k=1)\nelse:\n    [w, _] = np.linalg.eigh(h.todense())\nw\n"

The ansatz program is mostly the same as with pyquil when creating the circuit. 

In [24]:
 def var_ansatz(params):
    S = 2
    p = Program()
    
    # put initial state in the sector
    
    # Product state -- 1/sqrt(2) (|1001> + |0110>)
    state = QuantumStateGpu(4)
    c = QuantumCircuit(4)
    c.add_H_gate(0)
    c.add_CNOT_gate(0, 1)
    c.add_CNOT_gate(1, 2)
    c.add_X_gate(1)
    c.add_CNOT_gate(2, 3)
    c.add_X_gate(2)
    # |1001>
    #p.inst(X(0))
    #p.inst(X(3))
    
    # HF state -- 1/2 (|0101> + |0110> + |1001> + |1010>)

    for i in range(S):
        # building U_u
        c.add_RZ_gate(0, U1*params[3*i]/4)
        c.add_RZ_gate(1, U2*params[3*i]/4)
        c.add_RZ_gate(2, U1*params[3*i]/4)
        c.add_RZ_gate(3, U2*params[3*i]/4)
        c.add_CNOT_gate(0, 2)
        c.add_RZ_gate(2, -U1*params[3*i]/4)
        c.add_CNOT_gate(0, 2)
        c.add_CNOT_gate(1, 3)
        c.add_RZ_gate(3, -U2*params[3*i]/4)
        c.add_CNOT_gate(1, 3)
        
        #building U_\epsilon
        c.add_RZ_gate(1, e2*params[3*i+2])
        c.add_RZ_gate(3, e2*params[3*i+2])
        c.add_RZ_gate(0, e1*params[3*i+2])
        c.add_RZ_gate(2, e1*params[3*i+2])
        
        # building U_t
        c.add_H_gate(0)
        c.add_H_gate(1)
        c.add_H_gate(2)
        c.add_H_gate(3)
        c.add_CNOT_gate(0, 1)
        c.add_CNOT_gate(2, 3)
        c.add_RZ_gate(1, params[3*i+1])
        c.add_RZ_gate(3, params[3*i+1])
        c.add_CNOT_gate(0, 1)
        c.add_CNOT_gate(2, 3)
        c.add_H_gate(0)
        c.add_H_gate(1)
        c.add_H_gate(2)
        c.add_H_gate(3)
        c.add_RX_gate(0, -np.pi/2)
        c.add_RX_gate(1, -np.pi/2)
        c.add_RX_gate(2, -np.pi/2)
        c.add_RX_gate(3, -np.pi/2)
        c.add_CNOT_gate(0, 1)
        c.add_CNOT_gate(2, 3)
        c.add_RZ_gate(1, params[3*i+1])
        c.add_RZ_gate(3, params[3*i+1])
        c.add_CNOT_gate(0, 1)
        c.add_CNOT_gate(2, 3)
        c.add_RX_gate(0, -np.pi/2)
        c.add_RX_gate(1, -np.pi/2)
        c.add_RX_gate(2, -np.pi/2)
        c.add_RX_gate(3, -np.pi/2)
        
        # building U_u
        c.add_RZ_gate(0, U1*params[3*i]/4)
        c.add_RZ_gate(1, U1*params[3*i]/4)
        c.add_RZ_gate(2, U1*params[3*i]/4)
        c.add_RZ_gate(3, U1*params[3*i]/4)
        c.add_CNOT_gate(0, 2)
        c.add_RZ_gate(2, -U1*params[3*i]/4)
        c.add_CNOT_gate(0, 2)
        c.add_CNOT_gate(1, 3)
        c.add_RZ_gate(3, -U2*params[3*i]/4)
        c.add_CNOT_gate(1, 3)
    c.update_quantum_state(state)
        
    return c

Here I define the VQE with qulacs version. It is a lot different than with pyquil/grove, and I am not sure how to specify the optimization method yet to nelder-mead, which may be part of why the results are different.

In [25]:
def cost(x):
    state = QuantumStateGpu(n_qubits)
    circuit = var_ansatz(x)
    circuit.update_quantum_state(state)
    return hamiltonian.get_expectation_value(state)

This code is mostly the same as the other H2VQE code, with the adjustments for qulacs' VQE.

In [26]:
from scipy.optimize import minimize
import random

def expectation(x):
    """
    Expectation value for parameters, x
    """
    return cost(x)

def scipy_minimize(x):
    """
    Minimizes expectation value using parameters found in greedy search as starting point
    Powell's conjugate direction method
    """
    return minimize(expectation, x, method='powell')

def greedy_noisy_search(initial_state):
    """
    Slightly perturb the values of the points, accepting whenever this results in a lower energy
    Total of 150 evaulations of the energy
    Change step size after 30 evaluations based on number of acceptances in previous trial group
    """
    params = initial_state
    min_energy = cost(initial_state)

    def random_point(dim, step):
        """
        Generates a random point on the perimeter of a circle with radius, step
        """
        coords = [random.gauss(0, 1) for i in range(dim)]
        norm = math.sqrt(sum([i**2 for i in coords]))
        coords = [(i / norm) * step for i in coords]
        return coords

    step = 0.1
    acceptances = 0

    # Five groups of 30 trials
    for i in range(5):
        acceptances = 0
        for j in range(30):
            
            # Slightly perturb the values of the points
            coords = random_point(len(initial_state), step)
            temp_params = [sum(x) for x in zip(params, coords)]
            
            # Calculate expectation value with new parameters
            temp_energy = cost(temp_params)
            print(temp_params)
            
            # Greedily accept parameters that result in lower energy
            if (temp_energy < min_energy):
                min_energy = temp_energy
                params = temp_params
                acceptances += 1
        # Update step size
        step *= (acceptances / 15)

    return {'x': params, 'energy': min_energy, 'step': step}

def global_variational(S):
    """
    Complete a round of greedy and Powell's optimization for six randomly chosen points
    Further optimizes the best point
    """    
    points = [np.random.rand(S*3)*0.2 for i in range(6)]
    greedy_results = [greedy_noisy_search(x) for x in points]
    print("Done with greedy search")
    results = [scipy_minimize(i['x']) for i in greedy_results]
    print("Done with Powell's")
    res = min(results, key=lambda x:x.fun)
    print(res)
    # For the chosen point, we continue optimizing until we cannot find improvement
    res_copy = res
    energy = res_copy.fun
    while (True):
        result = greedy_noisy_search(res_copy.x)
        temp_res = scipy_minimize(result['x'])
        temp_energy = temp_res.fun
        print(temp_energy)

        # Optimizing will usually find an energy ~1e-6 lower
        # Stop it eventually
        tolerance = 0.00001
        if (abs(temp_energy - energy) > tolerance):
            res_copy = temp_res
            energy = temp_energy
        else:
            break
    return res_copy

In [27]:
# make sure to change this S to match the one in the ansatz
S = 2
res = global_variational(S)
print(res)

[0.1083913103645248, 0.0759094628101091, 0.11089309018720218, 0.0780612128013102, 0.02362798874947364, 0.1025470426156562]
[0.13991481049165336, 0.13702577312101072, 0.09377799087972026, 0.047953009920497724, 0.08603465511226883, 0.08924129039407398]
[0.1599658810173764, 0.0606148402944661, 0.17982433537344278, 0.04061205201656685, 0.008747717149879663, 0.12957960621575054]
[0.09697029841407911, 0.07781696292595887, 0.11337044414974948, 0.059657787858276634, -0.009285086277280849, 0.19440469384476952]
[0.1317516944008566, 0.05538622078677447, 0.10297916344323792, -0.009743973466429218, 0.057210828213479684, 0.11407530857547328]
[0.06462756165272775, 0.09123115578137651, 0.13312470924226288, 0.05407296060603584, -0.01661230695506867, 0.17438726460586493]
[0.07479503521680798, 0.09224410079196349, 0.027376409119604747, 0.06445496695300929, 0.019333889957214436, 0.14030766590410035]
[0.11147264822000844, 0.07635848948167268, 0.14988218656317942, 0.024792810914474273, 0.00840254514676246, 

[0.09075866459127484, 0.2364922817230411, 0.14807843195924597, 0.09908073132984227, 0.15997516362678083, 0.1698994626870018]
[0.10166236909451214, 0.11362897541006516, 0.16367607838352366, 0.026976556023379505, 0.20570170274090277, 0.11301234031792927]
[0.08868380960617703, 0.11546411671592625, 0.16837671060891762, 0.0227438452116719, 0.14866646709162612, 0.05631646369788531]
[0.04785523430788308, 0.1340925711388119, 0.1279442614485698, 0.03548029151005659, 0.17686821840476283, -0.017125141257223482]
[0.053120573002426986, 0.12837113484614865, 0.18729440195384023, 0.09330088618868099, 0.20334746480097593, 0.04074474261417775]
[0.07030143570921193, 0.0896513410714101, 0.14156195239209646, 0.09787309713913969, 0.19281021939091417, 0.08246378331136867]
[0.005255971178280319, 0.15421323065259807, 0.157931849063746, 0.0739515866895898, 0.21045371074611652, 0.061266024202625474]
[0.006073559104329607, 0.11032428318081489, 0.0791603780659581, 0.11079844709044855, 0.1584306624829222, 0.0967618

[0.638613793854532, 0.39305019474113123, -0.10238393611518178, 0.8040183506868653, -0.2749240492412, -0.1476445840766762]
[0.6880756083784326, 0.4041430816622343, -0.08850985453412326, 0.8580638974472047, -0.22119026792063085, -0.1198636250238871]
[0.6631525374794751, 0.42467199723913407, -0.11372024684734089, 0.8752910834258423, -0.2559985103331156, -0.14045792454895184]
[0.6419816564274976, 0.39020874030453884, -0.09035278487733761, 0.8528430011955801, -0.23419340346391737, -0.1616554462090661]
[0.6580307126425731, 0.4216427456267012, -0.1676032411109739, 0.8670715116258937, -0.27810878027751296, -0.15120327910366813]
[0.6367282243696845, 0.4156069028297328, -0.16335707087080226, 0.8877957971848993, -0.24207427398645157, -0.1444813291713882]
[0.661124936265997, 0.4110390197843209, -0.21164876119089265, 0.891683351554707, -0.23992859713647863, -0.1698349304725597]
[0.654491650111868, 0.400417706388341, -0.1988752411569658, 0.8490993677530337, -0.21759410916697974, -0.201115498579694]


[0.5236341544103142, 0.26270711075079745, -0.03887876677861232, 0.38774481914996856, -0.008457661784390289, 0.1476543185405058]
[0.5104653031840598, 0.22600454600264608, 0.028490903427264472, 0.3964111685602761, -0.0647216489251464, 0.0972104106058419]
[0.47908498952430745, 0.2899735899677597, -0.02480387163038217, 0.4175809651582654, 0.025807316622441536, 0.07570748700193221]
[0.4450184166647175, 0.24557569685274855, -0.0039213766365088845, 0.37854386944195373, 0.017018604742912332, 0.11079424810801461]
[0.5345491379396614, 0.3213832241762079, -0.056229711247525696, 0.3918013600621671, 0.00017635868125206317, 0.07041166979258362]
[0.5499178661150336, 0.29863639899086014, -0.08181312931415828, 0.34321077185406246, 0.04809326228479122, 0.08870915445376557]
[0.4935014406353058, 0.32750457915735487, -0.03893963289767242, 0.43105072656348636, 0.03688876907837971, 0.10901678476377258]
[0.5280750679552995, 0.34933258662788974, -0.04906115776231428, 0.4120721360333722, -0.07112240951178053, 0

[0.64075727158607, 0.3597950389194264, -0.24634387755935064, 0.8378781650422957, -0.12475448489016372, -0.043671314675607496]
[0.6263191970678276, 0.3700128834147046, -0.26276520684131754, 0.8463010042087239, -0.10598249143813332, -0.06832160814989056]
[0.6256499395060985, 0.3746520653389312, -0.2585723937128929, 0.8556910718579296, -0.09793725451920435, -0.0891568599933059]
[0.6294331281247626, 0.3581328091131093, -0.2638260294091448, 0.8593626297106717, -0.0935333817672994, -0.07245706775603208]
[0.6410031458846186, 0.37418575290393635, -0.24777138719468114, 0.8421948426775243, -0.09832973917792374, -0.09873047696728816]
[0.6374399446812181, 0.36216592920862506, -0.25266882049318795, 0.8444394460135906, -0.11773866592497594, -0.10670884926071168]
[0.6222550185560686, 0.3663148569448886, -0.25618846487105873, 0.8621296194320391, -0.11149203780537977, -0.1029127399783265]
[0.6242799942904583, 0.37627585132321123, -0.24879793678294806, 0.8808540218350709, -0.11962881449647064, -0.110110

...

[0.1846394508931546, 0.21117223982015787, -0.09176456619249977, 0.22869055602499083, -0.08536581506442387, 0.0021737946380541336]
[0.19772931736579114, 0.20886649063378984, -0.13908542443480698, 0.2734038200419349, -0.05850350965403208, -0.06756296654727381]
[0.2554730289094118, 0.238759533677294, -0.10809172133677514, 0.29253285556537123, -0.07752028065955605, -0.003657547813773865]
[0.19577180682500173, 0.1991544712742615, -0.10538541597938694, 0.31477980230404906, -0.012828287441773473, -0.017070168710124622]
[0.3102316393257104, 0.2780617379496383, -0.12058541806525111, 0.3325080565478461, -0.03178113654145792, 0.036475392128652695]
[0.292394442650033, 0.3214832654524165, -0.11513190397036639, 0.3802112168449625, 0.016557930757465665, -0.019688721132693522]
[0.30451502206343195, 0.31364653404120735, -0.13294823661310498, 0.42748228248376285, -0.05228072047596013, -0.023160471049010794]
[0.2947178322811445, 0.2984082259740117, -0.13991611935052248, 0.41505462345695554, 0.02947625258

[1.0202642332585068, 0.40335854581919434, 0.09323051472517231, 1.1108455607707102, -0.1968085335591557, -0.2426539291209913]
[0.9960619095689701, 0.39968259505734205, 0.09866299366020259, 1.1054759393082692, -0.18822329703460133, -0.2730378851761929]
[1.0168458015327575, 0.37249505438613884, 0.08890195884186597, 1.105934915146064, -0.21428110818941934, -0.26115037040452727]
[1.0364628033438215, 0.3796386916133871, 0.10050130401047519, 1.1190595463046003, -0.21948336724105333, -0.2566179416430258]
[1.0213716563962045, 0.3865268426414381, 0.0974951514914832, 1.107482945103769, -0.229951943940696, -0.2582099002990234]
[1.0157216269058482, 0.43657670658627135, 0.08309922834611216, 1.1165144825331035, -0.20082905756747468, -0.223173808700769]
[0.9931421230703256, 0.44743092124787004, 0.06428093359840842, 1.1353115166981402, -0.18389158845682738, -0.21747132854845805]
[0.9882065093855209, 0.4490819992106577, 0.08904081602664482, 1.1331489390340366, -0.19709768288216262, -0.2027996579286496]


[0.21907239727715966, 0.23247179026633874, 0.006360744806755626, 1.0742588332158565, 0.08238003219167962, -0.05242920761703329]
[0.2338891147527889, 0.2297606702191459, -0.03567104871279747, 1.0605143476606063, 0.11946886652641939, -0.05890613044955917]
[0.21224591939503418, 0.22903422159930306, 0.011037734268426608, 1.1311157381418282, 0.09127803121423257, -0.06684538140182622]
[0.21324853704479727, 0.2048115549938678, -0.010653870593667566, 1.123290064014119, 0.053578691067561975, -0.09939150307291968]
[0.20368776155196494, 0.23315452090272581, 0.008135316018326816, 1.1758504890798123, 0.053693211466186885, -0.05748907768143243]
[0.19602762106788396, 0.2593475607733085, -0.0011907798786966253, 1.1628824759970378, 0.0116568264350769, -0.028626579251410954]
[0.201396055076505, 0.24196804125922333, 0.026471630810163425, 1.1360256717210995, 0.040125443649357856, -0.019929137598620833]
[0.23091741767799442, 0.22328375704634476, -0.0076389702834238765, 1.1716265471017284, 0.093875746093868

Done with Powell's
   direc: array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [ 0.76086252, -0.00443009, -0.12775168, -0.04112326, -0.01517726,
        -0.03219141]])
     fun: array(-0.81082102)
 message: 'Optimization terminated successfully.'
    nfev: 347
     nit: 5
  status: 0
 success: True
       x: array([ 6.18210909,  0.4014311 ,  0.12032339,  0.20384   , -0.80350967,
       -0.4335439 ])
[6.159236562343292, 0.38563257733426776, 0.16099678227105385, 0.17948532886673751, -0.7295798590410518, -0.3946316598292966]
[6.235720393665538, 0.44235722504606034,

-0.8108210269654883
   direc: array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [ 0.76086252, -0.00443009, -0.12775168, -0.04112326, -0.01517726,
        -0.03219141]])
     fun: array(-0.81082102)
 message: 'Optimization terminated successfully.'
    nfev: 347
     nit: 5
  status: 0
 success: True
       x: array([ 6.18210909,  0.4014311 ,  0.12032339,  0.20384   , -0.80350967,
       -0.4335439 ])


The output here is the part that I am having trouble understanding. I am not sure what the nfev and nit variables represent. I believe the x array are the energy values, which are different than the pyquil output. Assuming all is running correctly, the execution of the global_variational function is much quicker when running on the GPU, which is a good sign. However, the output appears to be the ground state, so I think that it is not working.

In [19]:
print(res)

   direc: array([[-1.56582116e-02, -6.32288884e-03,  3.44269786e-01,
        -6.64508876e-02, -4.80328542e-03,  2.83880584e-02],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 4.04987070e-01,  1.28085204e-03,  2.84495090e-02,
        -2.84576853e-02,  1.70142438e-03, -1.15486379e-02],
       [ 1.03931003e-02,  6.69304008e-04,  4.82818653e-01,
        -5.41762825e-03, -2.26353988e-03,  8.60888284e-04]])
     fun: array(-0.81082103)
 message: 'Optimization terminated successfully.'
    nfev: 503
     nit: 7
  status: 0
 success: True
       x: array([ 6.69422356,  0.39490955,  4.12382597, -0.7936542 , -0.71572072,
       -0.31570506])


I have not gotten to redefining the rest of the code yet, as I want to make sure everything up to this point is running correctly.

In [10]:
def op(s1, s2):
    # returns a PauliSum representing a_s1^daggar a_s2
    def sigma_plus(s):
        # returns sigma^+ operator on side s
        # sigma^+ = 1/2(X + iY)
        return PauliTerm('X', s, 0.5) + PauliTerm('Y', s, complex(0.0+0.5j))
    def sigma_minus(s):
        # returns sigma^- operator on side s
        # sigma^- = 1/2(X - iY)
        return PauliTerm('X', s, 0.5) - PauliTerm('Y', s, complex(0.0+0.5j))
    
    # for all sites in between site one (s1) and site two (s2), multiply by sigma z
    z = sI()
    for i in range(s1+1, s2):
        z *= PauliTerm('Z', i)
        
    return sigma_minus(s1) * z * sigma_plus(s2)